In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

### importing data

In [2]:
run_qs = RunStats.objects.filter(psearch__id=83, method="BD", status=3)

pd.DataFrame.from_dict(run_qs.values('run_id', 'coherence', 'exclusivity', 'K', 'alpha',
                                'top_chain_var', 'rng_seed', 'max_iter'))

,K,alpha,coherence,exclusivity,max_iter,rng_seed,run_id,top_chain_var
0,40,0.05,NaN,NaN,50,1,2147,0.010
1,40,0.01,NaN,NaN,50,1,2146,0.010
2,45,0.01,NaN,NaN,50,1,2144,0.050
3,45,0.01,NaN,NaN,50,1,2145,0.010
4,45,0.01,NaN,NaN,50,1,2143,0.100
5,45,0.05,NaN,NaN,50,1,2136,0.010
6,45,0.10,0.374046,0.698753,50,1,2118,0.100
7,45,0.01,0.508955,0.742253,50,1,2111,0.005
8,45,0.10,0.491302,0.871996,50,1,2120,0.010
9,45,0.10,0.506084,0.780998,50,1,2121,0.005


In [3]:
run_id = 2111

save_dir = "./plots/topic_analysis_{}".format(run_id)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

dts = DocTopic.objects.filter(run_id=run_id)
# total score of all DocTopics
print(dts.aggregate(
    n = Sum('score')
))
# total score of DocTopics, sorted by topic
pd.DataFrame.from_dict(list(dts.values('topic__id', 'topic__title').annotate(
    totalscore = Sum('score')
).order_by('-totalscore')))

{'n': 4056676.39996124}


,topic__id,topic__title,totalscore
0,177264,"{darauf, darub, natur}",939731.269529
1,177261,"{regier, red, bundeskanzl}",286227.286681
2,177274,"{wirtschaft, bundesregier, sozial}",221517.472968
3,177241,"{bundesregier, hoh, staatssekretar}",171537.082003
4,177251,"{milliard, bund, rund}",141027.574810
5,177250,"{wirtschaft, wachstum, entwickl}",134348.092923
6,177242,"{europa, gemeinschaft, union}",130337.805275
7,177269,"{gesetz, wirtschaft, bundesregier}",112159.465356
8,177244,"{energi, strom, gas}",111039.675321
9,177249,"{million, tonn, kohl}",108805.956757


# party positions in topic space (tsne)

In [ ]:
# Runstats method: calculate_tsne

# compress folder to easily download it

In [ ]:
%%bash
zip -r plots/topic_analysis_2111.zip plots/topic_analysis_2111/